# Реализация алгоритма CBOW

## Импорты

In [ ]:
import random
from tqdm.notebook import tqdm

from nltk.util import ngrams
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader


## Сборка датасета

### Загрузка контекстных эмбеддингов

In [83]:
embeddings = pd.read_csv("/content/drive/MyDrive/Университет/NLP/compressed_embeddings200.csv")
embeddings = embeddings.set_index("word")
embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
word,,,,,,,,,,,,,,,,,,,,,
абп,-13.745661,6.144555,1.672390,-0.837235,-3.745957,-0.866193,-0.170243,0.518571,1.121641,-1.053889,...,-0.124310,-0.076910,0.048945,0.031750,0.007667,-0.038009,0.168864,0.000495,-0.130860,0.141327
абрамов,-3.991519,1.339566,11.473537,-3.064182,-5.942729,-3.422678,2.007564,2.223933,2.904329,-3.163823,...,-1.809141,-0.098477,-0.810195,-0.452103,-0.543618,1.680041,0.453972,-0.655151,-0.514539,-0.426249
абсолютно,-11.821876,2.943006,-1.280718,-0.987937,-1.371568,-1.522340,-0.233250,-0.602862,2.170925,-0.611029,...,0.308653,0.359587,0.387458,-0.257858,0.229763,-0.563574,0.312862,-0.277851,0.412213,-0.883354
авангардный,-7.512660,4.299288,8.587495,-1.755458,-5.053628,1.171524,0.330959,1.959238,3.094127,0.620284,...,-0.382019,0.042043,-0.119812,-0.030248,-0.457969,0.764405,0.292025,-0.466514,-0.159710,-0.276336
аварийка,-12.834715,5.345289,1.413563,-0.377485,-3.892651,-0.032718,0.037899,0.643811,1.148384,-1.069906,...,-0.308458,-0.167686,0.190215,0.444776,-0.127919,0.217852,0.118530,-0.164167,0.000987,-0.000986


### Применение функции Softmax к эмбеддингам

In [85]:
data = nn.Softmax(dim=1)(torch.from_numpy(embeddings.values).type(torch.float32))
embeddings = pd.DataFrame(data=data, index=embeddings.index)
embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
word,,,,,,,,,,,,,,,,,,,,,
абп,1.621377e-09,0.704848,0.008051,6.545494e-04,3.570265e-05,6.358665e-04,0.001275,0.002540,0.004642,5.270485e-04,...,0.001335,0.001400,0.001588,0.001561,0.001524,0.001456,0.001790,0.001513,0.001327,0.001742
абрамов,1.915329e-07,0.000040,0.996854,4.841507e-07,2.721721e-08,3.382887e-07,0.000077,0.000096,0.000189,4.382351e-07,...,0.000002,0.000009,0.000005,0.000007,0.000006,0.000056,0.000016,0.000005,0.000006,0.000007
абсолютно,3.006646e-08,0.077694,0.001138,1.524763e-03,1.038948e-03,8.935404e-04,0.003243,0.002241,0.035899,2.222755e-03,...,0.005576,0.005867,0.006033,0.003164,0.005153,0.002331,0.005599,0.003102,0.006184,0.001693
авангардный,9.628540e-08,0.012984,0.945734,3.047064e-05,1.125912e-06,5.689235e-04,0.000245,0.001251,0.003891,3.278334e-04,...,0.000120,0.000184,0.000156,0.000171,0.000112,0.000379,0.000236,0.000111,0.000150,0.000134
аварийка,6.490620e-09,0.510225,0.010005,1.668756e-03,4.963356e-05,2.355717e-03,0.002528,0.004634,0.007675,8.349835e-04,...,0.001788,0.002058,0.002944,0.003797,0.002142,0.003027,0.002740,0.002066,0.002436,0.002432


In [87]:
embeddings.sum(axis=1).head()

,0
word,
абп,1.0
абрамов,1.0
абсолютно,1.0
авангардный,1.0
аварийка,1.0


### Загрузка корпуса

In [45]:
corpus = pd.read_csv("/content/drive/MyDrive/Университет/NLP/corpus.csv")
corpus = corpus["corpus"].apply(lambda text: text.split())
corpus[:10]

,corpus
0,"[просить, убрать, всё, дерево, кустарник, кото..."
1,"[необходимо, проверить, законность, установка,..."
2,"[уборка, производиться, лестница, очень, грязн..."
3,"[отсутствовать, освещение, площадка, шесть, се..."
4,"[делать, благоустройство, никто, убирать, мусо..."
5,"[пожалуйста, удалить, бетонный, арматура, газо..."
6,"[ланский, дом, двенадцать, квартира, парадный,..."
7,"[кривой, висеть, просьба, поправить, этаж]"
8,"[плохой, уборка, улица, мусор, тротуар, газон,..."
9,"[демонтаж, рекламный, вывеска, фасад, образова..."


### Создание ngrams

In [94]:
df = []
len_ngrams = 5

for doc in corpus:
    df.extend(list(ngrams(doc, len_ngrams)))
display(df[:10], len(df))

[('просить', 'убрать', 'всё', 'дерево', 'кустарник'),
 ('убрать', 'всё', 'дерево', 'кустарник', 'который'),
 ('всё', 'дерево', 'кустарник', 'который', 'выйти'),
 ('дерево', 'кустарник', 'который', 'выйти', 'предел'),
 ('кустарник', 'который', 'выйти', 'предел', 'газон'),
 ('который', 'выйти', 'предел', 'газон', 'пешеходный'),
 ('выйти', 'предел', 'газон', 'пешеходный', 'зона'),
 ('предел', 'газон', 'пешеходный', 'зона', 'начинать'),
 ('газон', 'пешеходный', 'зона', 'начинать', 'подъезд'),
 ('пешеходный', 'зона', 'начинать', 'подъезд', 'тринадцать')]

341986

### Обрезка датасета

In [95]:
df = np.array(random.sample(df, 100_000))
y, x = df[:, 2].reshape(-1, 1), np.delete(df, 2, axis=1)
display(x[:10], y[:10], x.shape, y.shape)

array([['машина', 'приложить', 'отметить', 'одинаковый'],
       ['блок', 'лестница', 'иметь', 'весь'],
       ['этаж', 'грязь', 'песок', 'график'],
       ['свалить', 'мусор', 'работа', 'закончить'],
       ['каждый', 'день', 'лифт', 'нажать'],
       ['вода', 'напор', 'сильно', 'уменьшиться'],
       ['всё', 'плохой', 'топить', 'колодец'],
       ['наш', 'дом', 'перенос', 'площадка'],
       ['достижение', 'хороший', 'рекомендовать', 'производить'],
       ['резиновый', 'пустота', 'скотч', 'пусть']], dtype='<U20')

array([['скриншот'],
       ['балкон'],
       ['пыль'],
       ['газон'],
       ['вызывать'],
       ['вода'],
       ['топить'],
       ['обеспечить'],
       ['эффект'],
       ['заклеить']], dtype='<U20')

(100000, 4)

(100000, 1)

### Векторизация датасета

In [89]:
def word_vectorize(word: str) -> np.ndarray[float]:
    return embeddings.loc[word]


def document_vectorize(text: np.ndarray[str]) -> np.ndarray[float]:
    return np.apply_along_axis(word_vectorize, axis=0, arr=text).flatten()


def corpus_vectorize(corpus):
    return np.apply_along_axis(document_vectorize, axis=1, arr=corpus)


In [96]:
x = corpus_vectorize(x)
y = corpus_vectorize(y)
x.shape, y.shape

((100000, 800), (100000, 200))

### Разделение на тренировочную и тестовую выборку

In [97]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((80000, 800), (20000, 800), (80000, 200), (20000, 200))

### Создание DataLoader

In [98]:
train_ds = TensorDataset(
    torch.from_numpy(xtrain).type(torch.float32),
    torch.from_numpy(ytrain).type(torch.float32)
)
train_dl = DataLoader(train_ds, batch_size=256, shuffle=True)

## Нейронная сеть CBOW

In [103]:
class CBOW(nn.Module):
    def __init__(self, n_input: int, n_hidden: int, n_output: int) -> None:
        super(CBOW, self).__init__()
        self.input_layer = nn.Linear(n_input, n_hidden, dtype=torch.float32)
        self.hidden_layer = nn.Linear(n_hidden, n_output, dtype=torch.float32)
        self.output_layer = nn.Softmax(dim=1)

    def forward(self, X: torch.Tensor):
        y = self.input_layer(X)
        y = self.hidden_layer(y)
        y = self.output_layer(y)
        return y

    def fit(self, dataloader, loss, optimizer, batch_size: int = 256, n_epohs: int = 50) -> None:
        for epoh in tqdm(range(n_epohs), desc="Epoh"):
            for xbatch, ybatch in tqdm(dataloader, desc="Batch", leave=False):
                optimizer.zero_grad()
                outputs = self(xbatch)
                loss_value = loss(outputs, ybatch)
                loss_value.backward()
                optimizer.step()
            print(f" Эпоха: {epoh + 1}, loss: {loss_value.item()}")


## Обучение модели

In [104]:
model = CBOW(n_input=xtrain.shape[1], n_hidden=300, n_output=ytrain.shape[1])
print(model)

CBOW(
  (input_layer): Linear(in_features=800, out_features=300, bias=True)
  (hidden_layer): Linear(in_features=300, out_features=200, bias=True)
  (output_layer): Softmax(dim=1)
)


In [105]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=25*1e-4)

model.fit(train_dl, loss=loss, optimizer=optimizer)

Epoh:   0%|          | 0/50 [00:00<?, ?it/s]

Batch:   0%|          | 0/313 [00:00<?, ?it/s]

 Эпоха: 1, loss: 4.919835567474365


Batch:   0%|          | 0/313 [00:00<?, ?it/s]

 Эпоха: 2, loss: 5.0198140144348145


...

 Эпоха: 48, loss: 4.8789262771606445


Batch:   0%|          | 0/313 [00:00<?, ?it/s]

 Эпоха: 49, loss: 4.904966354370117


Batch:   0%|          | 0/313 [00:00<?, ?it/s]

 Эпоха: 50, loss: 4.89718770980835


## Сохранение модели

In [106]:
path = "/content/drive/MyDrive/Университет/NLP/CBOW_model.pth"
torch.save(model.state_dict(), path)

## Получение эмбеддингов

In [123]:
model_weights = model.state_dict()["hidden_layer.weight"]
model_weights.shape

torch.Size([200, 300])

In [124]:
embeddings.shape

(5190, 200)

In [130]:
new_embeddings = torch.from_numpy(embeddings.values).type(torch.float32) @ model_weights
new_embeddings.shape

torch.Size([5190, 300])

In [131]:
new_embeddings[:10]

tensor([[ 0.0576, -0.2071, -0.2203,  ..., -0.3126, -0.3441,  0.0861],
        [-0.0438, -0.0475, -0.1601,  ..., -0.3629, -0.0519, -0.1004],
        [-0.1149, -0.1147, -0.2160,  ..., -0.3608, -0.2149, -0.0813],
        ...,
        [ 0.0089, -0.2112, -0.2246,  ..., -0.2947, -0.2919,  0.0591],
        [ 0.1404, -0.2509, -0.2235,  ..., -0.2871, -0.4068,  0.1641],
        [-0.0435, -0.0474, -0.1600,  ..., -0.3629, -0.0515, -0.1005]])

In [134]:
np.savetxt("/content/drive/MyDrive/Университет/NLP/CBOW_embeddings.tsv", new_embeddings, delimiter="\t")